In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:48<00:00, 23.7MB/s]
100% 1.06G/1.06G [00:48<00:00, 23.3MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [6]:
#generators
train_ds= keras.utils.image_dataset_from_directory(
    directory="/content/train", labels='inferred',
    label_mode='int', batch_size=32, image_size=(227,227))

Found 20000 files belonging to 2 classes.


In [7]:
validation_ds=keras.utils.image_dataset_from_directory(
    directory="/content/test", labels='inferred',
    label_mode='int', batch_size=32, image_size=(227,227))

Found 5000 files belonging to 2 classes.


In [8]:
#Normalize image data
def process(image, label):
  image=tf.cast(image/255., tf.float32)
  return image, label

train_ds=train_ds.map(process)
validation_ds=validation_ds.map(process)

In [9]:
#create ALexNet
model = Sequential()
#frist conv2d
model.add(Conv2D(96, kernel_size=(11,11), strides=4, padding='valid', activation='relu', input_shape=(227,227,3)))
model.add(BatchNormalization())
#maxpooling
model.add(MaxPooling2D(pool_size=(3,3), strides=2))
#con2d 2nd
model.add(Conv2D(256, kernel_size=(5,5),padding='same', activation='relu'))
model.add(BatchNormalization())
#maxpooling
model.add(MaxPooling2D(pool_size=(3,3), strides=2))
#con2d 3rd
model.add(Conv2D(384, kernel_size=(3,3),padding='same', activation='relu'))
model.add(BatchNormalization())
#con2d 4
model.add(Conv2D(384, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
#con2d 5
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
#maxpooling
model.add(MaxPooling2D(pool_size=(3,3), strides=2))

#flatten
model.add(Flatten())

#DNN
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (Batch  (None, 55, 55, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Bat  (None, 27, 27, 256)       1024      
 chNormalization)                                                
                                                        

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(train_ds, epochs=5, validation_data=validation_ds)

Epoch 1/5
625/625 [==============================] - 52s 69ms/step - loss: 0.6538 - accuracy: 0.6402 - val_loss: 1.0948 - val_accuracy: 0.5130
Epoch 2/5
625/625 [==============================] - 40s 62ms/step - loss: 0.5308 - accuracy: 0.7345 - val_loss: 0.4880 - val_accuracy: 0.7598
Epoch 3/5
625/625 [==============================] - 39s 61ms/step - loss: 0.4217 - accuracy: 0.8041 - val_loss: 0.5964 - val_accuracy: 0.7670
Epoch 4/5
625/625 [==============================] - 40s 63ms/step - loss: 0.3455 - accuracy: 0.8488 - val_loss: 0.7597 - val_accuracy: 0.6176
Epoch 5/5
625/625 [==============================] - 43s 68ms/step - loss: 0.2961 - accuracy: 0.8728 - val_loss: 0.3660 - val_accuracy: 0.8394
